In [ ]:
import boto3
from file_extraction import *
from to_dataframe import *
import pandas as pd
import datetime, json

In [ ]:
s3_client = boto3.client('s3')
files_dict = {'academy_csv': [], 'json': [], 'txt': [], 'csv': []}

In [ ]:
# extract_file_type(s3_client, 'Talent', files_dict, 'txt')
# applicants = files_dict['txt']

# txt_df = convert_all_to_df(applicants, 'txt')

In [ ]:
# extract_file_type(s3_client, 'Talent', files_dict, 'json')
# applicants = files_dict['json']

# json_df = convert_all_to_df(applicants, 'json')

# json_df.to_csv('talent_json_all.csv', index=False)

In [ ]:
# json_file = s3_client.get_object(Bucket='data-eng-228-final-project', Key='Talent/10401.json')
# json_df = json_to_pd(json_file)
# json_df


In [ ]:
# extract_file_type(s3_client, 'Talent', files_dict, 'csv')
# applicants = files_dict['csv']

# csv_df = convert_all_to_df(applicants, 'csv')


In [ ]:
# duplicated_names = csv_df[csv_df.duplicated(subset='name', keep=False)]
# print(duplicated_names)

In [ ]:
# extract_file_type(s3_client, 'Academy', files_dict, 'csv')
# applicants = files_dict['academy_csv']

# ac_csv_df = convert_all_to_df(applicants, 'csv')
# ac_csv_df

In [ ]:
# duplicated_names = ac_csv_df[ac_csv_df.duplicated(subset='name', keep=False)]
# print(duplicated_names)

In [ ]:
# Extract names and dates from the Academy .csv files describing name of the course and date
def course_names() -> dict:
    course_table = []

    # List the objects in the S3 bucket
    response = s3_client.list_objects(Bucket="data-eng-228-final-project", Prefix="Academy")

    # Iterate through the objects in the S3 bucket
    for obj in response['Contents']:
        file_key = obj['Key']
        
        # Extract the course name from the file key
        file_name = file_key.split('/')[-1].split('.')[0].split('_')[:2]
        course_name = file_name[0] + '_' + file_name[1]
        course_group = file_name[0]
        course_date = file_key.split('/')[-1].split('.')[0].split('_')[-1:]
        course_table.append([course_name, course_group, course_date[0]])
    

        course_df = pd.DataFrame(course_table, columns=['course_name', 'course_group', 'course_date'])
    
    return course_df

course_names()

In [ ]:
def academy_table():
    # Initialize an empty list to store data
    course_table = []

    # List the objects in the S3 bucket
    response = s3_client.list_objects(Bucket="data-eng-228-final-project", Prefix="Academy")

    # Iterate through the objects in the S3 bucket
    for obj in response['Contents']:
        file_key = obj['Key']
        
        # Extract the course name from the file key
        file_name = file_key.split('/')[-1].split('.')[0].split('_')[:2]
        course_name = file_name[0] + '_' + file_name[1]
        course_group = file_name[0]
        course_date = file_key.split('/')[-1].split('.')[0].split('_')[-1:]
        
        # Read the CSV file from S3 and extract student IDs
        csv_object = s3_client.get_object(Bucket="data-eng-228-final-project", Key=file_key)
        df = pd.read_csv(csv_object['Body'])
        student_ids = df['name'].tolist()
        
        # Append the data to the list
        course_table.extend([(student_id, course_name, course_date[0]) for student_id in student_ids])

    # Create the DataFrame
    course_df = pd.DataFrame(course_table, columns=['name', 'course_name', 'course_date'])

    # Return the DataFrame
    return course_df

academy_table()


In [ ]:
academy_table()

In [ ]:
# Amy's code for courses table
def create_courses(dataframe):
    courses_df  = dataframe
    courses_df = pd.DataFrame().assign(course = courses_df['course_name'], date = courses_df['course_date'])
    courses_df['course_id'] = courses_df.groupby(['course']).ngroup()
    courses_df = courses_df[['course', 'course_id', 'date']].copy()
    courses_df.set_index('course_id', inplace=True)
    return courses_df



create_courses(course_names())

In [ ]:
courses = create_courses(course_names())
talent_all_json = pd.read_csv("partial_talent.csv")

def academy_table_df(courses_df, academy_table_df):
    academy = []

    for index, row in academy_table_df.iterrows():
        # Assigning variables to values in candidate table
        inv_date = row['date']
        interest = row['course_interest']
        # 'name' to be changed to 'name_iD' from the candidates table
        candidate_id = row['name']
        
        if row['result'] == 'Pass':
            # Setting conditions to choose the courses after candidates invitation date
            # Grouping by interest
            start_date_condition = courses_df['date'] > inv_date
            course_name_condition = courses_df['course'].str.split('_').str[0] == interest
            sorted_dates = courses_df[course_name_condition & start_date_condition]
            candidate_course = sorted_dates.sort_values('date', ascending=False)
            academy.append([candidate_id, candidate_course])
            academy_df = pd.DataFrame(academy, columns=['name_id', 'course'])
            

    # merged_df = pd.merge(courses_df, academy_df, on='course', how='left')
    # merged_df.drop('course', axis=1, inplace=True)
    # merged_df.rename(columns={'course_id': 'course'}, inplace=True)

    return academy


print(academy_table_df(courses, talent_all_json))

    